# UMAP-HDBSCAN on simulated populations

This notebook is a companion to the simulation data provided at `10.5281/zenodo.17545804`. The simulations were used as experiments described in [Diaz-Papkovich et al (2025)](https://www.biorxiv.org/content/10.1101/2023.07.06.548007v2).

For code used to generate UMAP embeddings, see the Topstrat repo: https://github.com/diazale/topstrat

## Convert vcf to numpy

In [2]:
import allel
import os

In [3]:
base_dir = "popsim_data"
data_dir = os.path.join(base_dir, "sims")
out_dir = os.path.join(base_dir, "numpy")

## Libraries and helper functions

In [4]:
import umap
import hdbscan
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import time

from collections import defaultdict

from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import v_measure_score
from sklearn.metrics import fowlkes_mallows_score

In [5]:
# Extract UMAP parameters from a file name
# e.g. "UMAP_NC5_NN50_MD0_2025-10-09_american_admix_"
def extract_umap_params(instr_):
    # Intake: A UMAP filename formatted that I can extract the UMAP parameters
    # Output: number components, number neighbours, minimum distance
    nc_ = instr_.split("_NC")[1].split("_")[0]
    nn_ = instr_.split("_NN")[1].split("_")[0]
    md_ = instr_.split("_MD")[1].split("_")[0]

    return nc_, nn_, md_

In [6]:
# Function to plot data
# Plots the first two dimensions of coords_ regardless of the matrix
def plotter(coords_, labels_, title_, size_=1, tick_params_=False):
    for label_ in set(labels_) - {-1}:
        plt.scatter(
            coords_[label_==labels_,0],
            coords_[label_==labels_,1],
            s=size_
        )
    plt.scatter(
        coords_[labels_==-1,0],
        coords_[labels_==-1,1],
        s=size_,
        c="black"
    )

    if tick_params_:
        plt.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
    else:
        plt.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)
            
    plt.title(title_)

In [7]:
# Look at how parameters affect which points get assigned to clusters
def four_split(in_proj, eps_list=[0.5], mps_list=[50, 25, 15, 10], labels=False, model_label="Unspecified"):

    fig, axs = plt.subplots(2,2, figsize=(10,10))
    fig = plt.figure(figsize=(10,10))
    
    #eps = [0.5]
    #mps = [50, 25, 15, 10]
    
    dim1 = 0
    dim2 = 1
    
    # Add HDBSCAN data
    clusterers = list()
    cluster_label_list = list()
    num_clusters = list()
    num_unclustered = list()
    
    for m in mps_list:
        for e in eps_list:
            clusterer = hdbscan.HDBSCAN(min_cluster_size=m, cluster_selection_epsilon=e).fit(in_proj)
            clusterers.append(clusterer)
            cluster_labels = clusterer.labels_
            cluster_label_list.append(cluster_labels)
            
            num_clusters.append(len(set(cluster_labels[cluster_labels!=-1])))
            num_unclustered.append(np.sum(cluster_labels==-1))

            # Arrange the four grids
            # Three possibilities: 4 epsilon values, 1 mp; 2 of each; 1 epsilon, 4 mp
            # overly fancy way to determine grid position from index in a 1D array.
            if len(mps_list)==4:
                # One epsilon value, four MP values
                xa = int(np.where(np.array(mps_list)==m)[0][0]/2)
                ya = np.where(np.array(mps_list)==m)[0][0] % 2
            elif len(eps_list)==4:
                # Four epsilon values, one MP value
                xa = int(np.where(np.array(eps_list)==e)[0][0]/2)
                ya = np.where(np.array(eps_list)==e)[0][0] % 2
            elif len(mps_list)==2 and len(eps_list)==2:
                xa = np.where(np.array(mps_list)==m)[0][0] % 2
                ya = np.where(np.array(eps_list)==e)[0][0] % 2
            #    print()

            if not labels:
                # Plot all points. Those assigned to a cluster are black, those not are red.
                axs[xa, ya].scatter(
                    in_proj[:,dim1],
                    in_proj[:,dim2],
                    s = 3
                )
                axs[xa, ya].scatter(
                    in_proj[cluster_labels==-1,dim1],
                    in_proj[cluster_labels==-1,dim2],
                    s = 3,
                    c="red"
                )
                axs[xa, ya].set_title("(MP, EPS)=(" + str(m) + ", " + str(e) + ")" + "\n" + \
                                     "unclustered=" + str(num_unclustered[-1]) + ", " + \
                                     "num_clusters=" + str(num_clusters[-1]))
            else:
                # Plot the labels specified by HDBSCAN
                for cl in (set(cluster_labels)):
                    if cl!=-1:
                        axs[xa, ya].scatter(
                        in_proj[cluster_labels==cl,dim1],
                        in_proj[cluster_labels==cl,dim2],
                        s = 3
                )
                axs[xa, ya].scatter(
                    in_proj[cluster_labels==-1,dim1],
                    in_proj[cluster_labels==-1,dim2],
                    s = 5,
                    c="black",
                    marker="o"
                )
                axs[xa, ya].set_title("(MP, EPS)=(" + str(m) + ", " + str(e) + ")" + "\n" + \
                             "unclustered=" + str(num_unclustered[-1]) + ", " + \
                             "num_clusters=" + str(num_clusters[-1]))
                axs[xa, ya].set_xticks([])
                axs[xa, ya].set_yticks([])

# Example clustering and analysis

## Set up directories and known population labels

This assumes you have already run UMAP on each of the simulated replicates. From here, it will import UMAP and run HDBSCAN. It also supports PCA and k-means if you want to compare. Otherwise, comment those portions of code out.

In [8]:
base_dir = "popsims"

# The three experiments are stored in:
# american_admix
# stepping_stone
# neutral_ooa_simulations

data_dir = {}

data_dir["ooa_neutral"] = os.path.join(base_dir, "neutral_ooa_simulations")
data_dir["stepping_stone"] = os.path.join(base_dir, "stepping_stone")
data_dir["admix"] = os.path.join(base_dir, "american_admix")

In [9]:
# Set up some arrays of ground truth populations as defaultdicts of lists
ground_truths = defaultdict(list)

ground_truths["ooa_neutral"] = np.array(
    [0]*1000 + [1]*1000 + [2]*1000
)
ground_truths["stepping_stone"] = np.array(
    [0]*500 + [1]*500 + [2]*500 + [3]*500 + [4]*500 + [5]*500 + [6]*500 + [7]*500 + [8]*500
)
ground_truths["admix"] = np.array(
    [0]*1000 + [1]*1000 + [2]*1000 + [3]*1000
)

In [10]:
umap_path = os.path.join(data_dir["ooa_neutral"], "embeddings", "umap")
pca_path = os.path.join(data_dir["ooa_neutral"], "embeddings", "pca")

ground_truth = ground_truths["ooa_neutral"]

In [ ]:
# This loops through the PCA and UMAP embeddings of the 100 simulated populations.
umap_embeddings = dict()
pca_embeddings = dict()

# HDBSCAN parameters (or UMAP parameters if necessary)
es = [0.5]
mps = [25,50]

hdbscan_clusters = dict()
kmeans_clusters = dict()

# For HDBSCAN
num_clusters = dict()
num_unclustered = dict()

# Cluster diagnostics versus ground truth labels
kmeans_ari = []
kmeans_vm = []
kmeans_fw = []

hdbscan_ari = defaultdict(list)
hdbscan_vm = defaultdict(list)
hdbscan_fw = defaultdict(list)

# Set up a variable to flag the highest cluster metric for each replicate.
# This works across grid searches if you're interested in which parameter sets do well.
max_ari = defaultdict(int)
max_vm = defaultdict(int)
max_fw = defaultdict(int)

correct_count = defaultdict(bool)

# Assumes your files are stored as [base_file]_[rep].npz with data stored in array ["arr_0"]
base_file = "UMAP_NC2_NN50_MD0.0001_2025-09-30_neutral_simulation_" 

for rep in range(1,101):
    print("Replicate:", rep)
    umap_embeddings[rep] = np.load(os.path.join(umap_path, base_file + str(rep) + ".npz"))["arr_0"]
    pca_embeddings[rep] = np.load(os.path.join(pca_path, "neutral_simulation_" + str(rep) + "_PCA.npz"))["arr_0"][:,:10]

    # K-means, for comparison
    kmeans = sklearn.cluster.KMeans(n_clusters=len(set(ground_truth))).fit(pca_embeddings[rep][:,:len(set(ground_truth))])
    kmeans_clusters[rep] = kmeans.labels_

    kmeans_ari.append(adjusted_rand_score(ground_truth, kmeans_clusters[rep]))
    kmeans_vm.append(v_measure_score(ground_truth, kmeans_clusters[rep]))
    kmeans_fw.append(fowlkes_mallows_score(ground_truth, kmeans_clusters[rep]))
    
    for e in es:
        for m in mps:
            # Generate HDBSCAN clusters
            clusterer = hdbscan.HDBSCAN(min_cluster_size=m, cluster_selection_epsilon=e).fit(umap_embeddings[rep])
            hdbscan_clusters[(rep, e, m)] = clusterer.labels_
            num_clusters[(rep, e, m)] = len(set(hdbscan_clusters[(rep, e, m)][hdbscan_clusters[(rep, e, m)]!=-1]))
            num_unclustered[(rep, e, m)] = np.sum(hdbscan_clusters[(rep, e, m)]==-1)

            # Various clustering metrics
            hdbscan_ari[(rep, e, m)].append(adjusted_rand_score(ground_truth, hdbscan_clusters[(rep, e, m)]))
            hdbscan_vm[(rep, e, m)].append(v_measure_score(ground_truth, hdbscan_clusters[(rep, e, m)]))
            hdbscan_fw[(rep, e, m)].append(fowlkes_mallows_score(ground_truth, hdbscan_clusters[(rep, e, m)]))

            # Check if this is the highest clustering metric for the replicate
            # Useful in a grid search to see if any parameters do particularly well
            max_ari[rep] = max(max_ari[rep], adjusted_rand_score(ground_truth, hdbscan_clusters[(rep, e, m)]))
            max_vm[rep] = max(max_vm[rep], v_measure_score(ground_truth, hdbscan_clusters[(rep, e, m)]))
            max_fw[rep] = max(max_fw[rep], fowlkes_mallows_score(ground_truth, hdbscan_clusters[(rep, e, m)]))

            # Check if any set of parameters in this replicate gets the number of clusters right
            if num_clusters[(rep, e, m)]==len(set(ground_truth)):
                correct_count[rep] = True

In [ ]:
# Find the highest and lowest HDBSCAN ARI w.r.t. population labels
min_ari = 1
max_ari = 0

for k in hdbscan_ari.keys():
    ari_val = hdbscan_ari[k][0]
    if ari_val > max_ari:
        max_key = k
        max_ari = ari_val
    if ari_val < min_ari:
        min_key = k
        min_ari = ari_val

max_rep = max_key[0]
min_rep = min_key[0]

In [ ]:
# Plot the highest ARI 
plotter(umap_embeddings[max_rep], ground_truth, "Highest-ARI UMAP-HDBSCAN (colours are ground truth)", size_=3)

In [ ]:
plotter(umap_embeddings[max_rep], temp, "Highest-ARI UMAP-HDBSCAN (colours are clusters)", size_=3)

In [ ]:
# Plot the lowest ARI
plotter(umap_embeddings[min_rep], ground_truth, "Lowest-ARI UMAP-HDBSCAN (colours are ground truth)", size_=3)

In [ ]:
plotter(umap_embeddings[min_rep], hdbscan_clusters[min_key], "Lowest-ARI UMAP-HDBSCAN (colours are clusters)", size_=3)

In [ ]:
# Look at the embedding with the lowest HDBSCAN ARI.
# Try a grid search of HDBSCAN parameters to see what (if anything) changes.
k = min_key
four_split(umap_embeddings[k[0]], mps_list=[25], eps_list=[0.5, 0.3, 0.2, 0.1], labels=True)